In [3]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [6]:
data.shape

(1160, 6)

In [7]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,52.651001,53.799000,52.651001,53.660500,53.660500,31766000
2018-01-03,53.696499,54.805000,53.671501,54.576000,54.576000,31318000
2018-01-04,54.854500,55.203999,54.713001,54.787998,54.787998,26052000


In [8]:
data.sort_index(inplace = True)

In [9]:
data = data.loc[~data.index.duplicated(keep='first')]

In [10]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-08-08,118.389999,120.150002,116.879997,117.300003,117.300003,19138300
2022-08-09,117.139999,117.320000,115.709999,116.629997,116.629997,19086500
2022-08-10,118.779999,120.919998,118.410004,119.699997,119.699997,30887100


In [11]:
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [12]:
import plotly.graph_objects as go

In [13]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [14]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [15]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,53.660500,31766000
2018-01-03,54.576000,31318000
2018-01-04,54.787998,26052000


In [16]:
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [17]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [18]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/1128 [00:00<?, ?it/s]

In [19]:
X.shape , Y.shape

((1128, 32, 2), (1128,))

In [20]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [21]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((639, 32, 2), (639,))

In [22]:
Xtest.shape , Ytest.shape

((489, 32, 2), (489,))

In [23]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [24]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [25]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [26]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [27]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

In [28]:
#define the callbacks for our model.
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [30]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [31]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
639/639 [==============================] - 111s 161ms/step - loss: 0.0119 - val_loss: 1.3288 - lr: 0.0020
Epoch 2/10
639/639 [==============================] - 102s 159ms/step - loss: 0.0127 - val_loss: 0.9748 - lr: 0.0020
Epoch 3/10
639/639 [==============================] - 101s 158ms/step - loss: 0.0106 - val_loss: 0.8165 - lr: 0.0020
Epoch 4/10
639/639 [==============================] - 102s 159ms/step - loss: 0.0076 - val_loss: 0.6328 - lr: 0.0020
Epoch 5/10
639/639 [==============================] - 102s 160ms/step - loss: 0.0068 - val_loss: 0.5222 - lr: 0.0020
Epoch 6/10
639/639 [==============================] - 101s 158ms/step - loss: 0.0062 - val_loss: 0.4413 - lr: 0.0020
Epoch 7/10
639/639 [==============================] - 102s 160ms/step - loss: 0.0055 - val_loss: 0.3947 - lr: 0.0020
Epoch 8/10
639/639 [==============================] - 102s 160ms/step - loss: 0.0053 - val_loss: 0.3420 - lr: 0.0020
Epoch 9/10
639/639 [==============================] - 100s 157ms

In [32]:
model.load_weights("best_weights.h5")

In [33]:
Predictions = model.predict(Xtest)

In [34]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [35]:
Predictions.shape

(489, 1)

In [36]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [38]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [40]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [41]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [42]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [44]:
Predictions = model.predict(Total_features)

In [45]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [46]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [47]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()
